In [ ]:
!pip install Pyfhel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Pyfhel: filename=Pyfhel-3.4.2-cp310-cp310-linux_x86_64.whl size=8500172 sha256=39060269627a1cc07e1e06c0ea4de91652e21ac88cd7a91d504b90e00f0f9299
  Stored in directory: /root/.cache/pip/wheels/74/7b/34/876b6d79c02f65e68e4e6f262735c2347cf77581ac50ca470d
Successfully built Pyfhel


In [ ]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt
import pickle
import numpy as np
from PIL import Image
import io
import requests
import zlib
from IPython.display import display
import wave

# Initialize Pyfhel

In [ ]:
HE = Pyfhel()
HE.contextGen(scheme='BFV', n=2**14, t_bits=20)
HE.keyGen()

In [ ]:
input_wav = "/content/Spiderman_5.wav"

# Get the Params and Frames

In [ ]:
with wave.open(input_wav, 'rb') as wav_file:
    params = wav_file.getparams()
    frames = wav_file.readframes(wav_file.getnframes())

# Serialize Data

In [ ]:
serialized_data = pickle.dumps((params, frames))

In [ ]:
Text = serialized_data.hex()

In [ ]:
print(Text)

...


# Encryption

In [ ]:
encoded_text = [ord(char) for char in Text]

In [ ]:
cipher_text = [HE.encryptInt(np.array([num], dtype=np.int64)) for num in encoded_text]
print(f"Cipher Text: {[str(ct) for ct in cipher_text]}")

In [ ]:
decrypted_text = [HE.decryptInt(ct)[0] for ct in cipher_text]
decoded_text = ''.join([chr(num) for num in decrypted_text])

print(f"Original Text: {Text}")
print("")
print(f"Decoded Text after encryption/decryption: {decoded_text}")

In [ ]:
decoded_bytes = bytes.fromhex(decoded_text)

In [ ]:
loaded_params, loaded_frames = pickle.loads(decoded_bytes)

In [ ]:
with wave.open("Decrypted_Audio.wav", 'wb') as wav_file:
    wav_file.setparams(loaded_params)
    wav_file.writeframes(loaded_frames)